# Create a SQLite3 schema to store Spotify data in at least 3rd normal form (3NF), and populate the tables. 

Data source: [https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md)

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
%load_ext sql

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')

## Normalization

- Every table should not have any 
    - duplication
    - dependencies that are not key or domain constraints

In [4]:
## check duplicates
df.duplicated().any()

False

There seem to be no duplicate rows in the data frame.

### 1NF

- Split composite entries

In [5]:
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


The data frame does not seem to have any compound entries, so the data frame already satisfies the first normal form.


### 2NF

- Break partial dependencies
    - Identify candidate Primary Key for each row
    - If there is a composite Primary Key, see if other columns have partial dependencies

In [6]:
df.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

In the codebook, `track_id` is the unique ID for each song, so it is possibly the Primary Key.

#### (1) Playlist: 

In [7]:
playlist = df[df.duplicated(subset=['playlist_id'])]
playlist.sort_values(by = ['playlist_id'], axis=0)[['playlist_id','playlist_name','playlist_genre','playlist_subgenre']].head()

,playlist_id,playlist_name,playlist_genre,playlist_subgenre
7949,0275i1VNfBnsNbPl0QIBpG,Hip-Hop 'n RnB,rap,southern hip hop
7935,0275i1VNfBnsNbPl0QIBpG,Hip-Hop 'n RnB,rap,southern hip hop
7936,0275i1VNfBnsNbPl0QIBpG,Hip-Hop 'n RnB,rap,southern hip hop
7937,0275i1VNfBnsNbPl0QIBpG,Hip-Hop 'n RnB,rap,southern hip hop
7938,0275i1VNfBnsNbPl0QIBpG,Hip-Hop 'n RnB,rap,southern hip hop


From the above table, we can see that `playlist_name`, `playlist_genre` and `playlist_subgenre` all depend on `playlist_id`. 

In [8]:
df_playlist = df[['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre']].drop_duplicates()

#### (2) Album: 

In [9]:
album = df[df.duplicated(subset=['track_album_id'])]
album.sort_values(by = ['track_album_id'], axis=0)[['track_album_id', 'track_album_name', 'track_album_release_date']].head()

,track_album_id,track_album_name,track_album_release_date
25103,006AgHXrEw13oyg0D8evRa,Bad Boy Greatest Hits Vol. 1,1998
11218,007DWn799UWvfY1wwZeENR,i am > i was,2018-12-21
11244,007DWn799UWvfY1wwZeENR,i am > i was,2018-12-21
11242,007DWn799UWvfY1wwZeENR,i am > i was,2018-12-21
8985,007DWn799UWvfY1wwZeENR,i am > i was,2018-12-21


From the above table, we can see that `track_album_name` and `track_album_release_date` all depend on `track_album_id`. 

In [10]:
df_album = df[['track_album_id','track_album_name', 'track_album_release_date']].drop_duplicates()

#### (3) Song: 

In [11]:
song = df[df.duplicated(subset=['track_id'])]
song.sort_values(by = ['track_id'], axis=0)[['track_id','track_name', 'track_artist','track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                             'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms']].head()

,track_id,track_name,track_artist,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
32084,00Gu3RMpDW2vO9PjlMVFDL,Hide Away (feat. Envy Monroe),Blasterjaxx,42,0.573,0.746,10,-4.894,1,0.0421,0.02490,0.000000,0.3610,0.134,130.001,188000
23850,00QyLmjxaSEE8qIZQjBXBj,We Own It (Fast & Furious),2 Chainz,59,0.554,0.899,8,-4.573,1,0.4080,0.05210,0.000000,0.0568,0.552,171.966,227893
28968,00QyLmjxaSEE8qIZQjBXBj,We Own It (Fast & Furious),2 Chainz,59,0.554,0.899,8,-4.573,1,0.4080,0.05210,0.000000,0.0568,0.552,171.966,227893
9345,00ReeHCY0FQUyuAUyPJdnk,Ain't No Future In Yo' Frontin',MC Breed,48,0.672,0.761,0,-9.644,1,0.2480,0.05140,0.000000,0.4740,0.731,102.192,244733
14626,00WIXhVVhswHuS6dlkScuw,Hot,Confetti,51,0.607,0.908,7,-3.883,1,0.1070,0.00297,0.000002,0.1200,0.664,168.015,150714


From the above table, we can see that `track_name`, `track_popularity` and `track_artist` and other song characteristics all depend on `track_id`. 

In [12]:
df_song = df[['track_id','track_name','track_artist','track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].drop_duplicates()

In [13]:
df_song_playlist = df[['track_id', 'playlist_id']].drop_duplicates()
df_song_album = df[['track_id', 'track_album_id']].drop_duplicates()

### 3NF

- Remove transitive dependencies

In [14]:
df_playlist_ = df_playlist[['playlist_id', 'playlist_name', 'playlist_subgenre']]
df_playlist_genre = df_playlist[['playlist_subgenre', 'playlist_genre']].drop_duplicates()

In [15]:
df_playlist_genre

,playlist_subgenre,playlist_genre
0,dance pop,pop
1298,post-teen pop,pop
2427,electropop,pop
3835,indie poptimism,pop
5507,hip hop,rap
6829,southern hip hop,rap
8504,gangster rap,rap
9962,trap,rap
11253,album rock,rock
12318,classic rock,rock


## Final tables

In [16]:
df_song.sample(5)

,track_id,track_name,track_artist,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
19218,4l3xGFwoTinvYxsePytOOf,Báilame - Remix,Nacho,69,0.646,0.865,6,-4.215,1,0.0983,0.2690,0.000004,0.1100,0.762,191.954,217093
32446,0XBpFhAb7DokSAhDvCBNY4,Double Double,twoloud,28,0.689,0.984,4,-3.336,1,0.1170,0.0163,0.186000,0.2900,0.415,123.982,166431
26668,4GhbRPmkVfAZmSQZqdifrn,ifuleave [feat. Mary J. Blige],Musiq Soulchild,45,0.467,0.752,11,-3.244,1,0.2320,0.2080,0.000000,0.1640,0.728,122.775,252440
20617,5bLrBRYR92X7CddgbVhUqc,Vamos a la Playa,Righeira,0,0.846,0.853,8,-6.440,1,0.0379,0.0179,0.048600,0.0392,0.662,128.534,218651
16468,2cElznINAHvKeHzv97ZuAV,Caramelo,La Lá,45,0.629,0.189,9,-14.598,1,0.0682,0.9720,0.006590,0.1080,0.441,118.402,203808


In [17]:
df_song_album.sample(5)

,track_id,track_album_id
3458,5TQnwwwx5na4rCj6EVcdvV,6gHRLG5Gbjk3vwtgmadx1g
8502,0yaMn1bb6b5gJwR3eVudOK,14jLi7wbq1YQEGMEUzpIkK
29672,1oSLMJeccvgqel2kyhdd9j,7zfC6BBJDNGfWXOSH9BvR3
10653,3G79AJm9H5lKnngSgBT21L,26WTFLXc2Y8ps9P4QuLr27
6479,7nn3FDp3e8h2LlNE25nXfH,7kyRRp16g59X6Zm4ID3NjL


In [18]:
df_song_playlist.sample(5)

,track_id,playlist_id
28525,1CYAEcbhLAluTm47dIFFgZ,5Bx5niVgi3qGQQw06C0RKq
13185,6YnhY4SuFyx3gMgeyTlVdI,37i9dQZF1DWVyizF9BJ61m
11233,4kfEy7h6bWBS5OaDpccTap,7tkgK1tm9hYkWp7EFyOcAr
17127,4sx5kcyZVyRd1okZAJQ7jX,3YdAwfgQLMS4AN2WjRfRlb
4737,1ywuaylLSmjHXsZOr0CY9U,5qFXOOxrQVyS4UCq3UilZN


In [19]:
df_playlist_.sample(5)

,playlist_id,playlist_name,playlist_subgenre
24337,3krpccUV68nBGAQbvHEZDC,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",new jack swing
18018,4BmnnqWEQAamNoVjtjrQJP,Latin Pop antiguo,latin pop
26790,317O0e8iWJLClLGDKtieRe,Electro House 2020,electro house
29320,6pPpLz42hUkr8gbMirkAEl,ALPAS Music Festival,big room
2411,4TvZA7Pml7mHLgbwNomlnm,post-teen pop,post-teen pop


In [20]:
df_playlist_genre.sample(5)

,playlist_subgenre,playlist_genre
25153,neo soul,r&b
0,dance pop,pop
3835,indie poptimism,pop
26790,electro house,edm
2427,electropop,pop


In [21]:
df_album.sample(5)

,track_album_id,track_album_name,track_album_release_date
814,0CADmCXbIx4F9m6TBwLtFd,'N Sync,1997-05-26
1284,5tF2lAa2rh2kU2xIiBzWia,Wanted on Voyage,2014-06-27
6076,3SZr5Pco2oqKFORCP3WNj9,Graduation,2007-01-01
1175,1aevseu4cRmSeh5OQOLy4J,Help,2019-11-22
17118,2DFTa7ls0ZDZKXT2UJsx5u,Praise You,2019-01-18


## Store it in Sqlite3

In [22]:
# Create a database to connect to in memory
cr = sqlite3.connect('spotify.db')

# Store dataframes in the database
df_song.to_sql(name='df_song', con=cr, index = False)
df_song_album.to_sql(name='df_song_album', con=cr, index = False)
df_song_playlist.to_sql(name='df_song_playlist', con=cr, index = False)
df_playlist_.to_sql(name='df_playlist', con=cr, index = False)
df_playlist_genre.to_sql(name='df_playlist_genre', con=cr, index = False)
df_album.to_sql(name='df_album', con=cr, index = False)

## Connect 

In [23]:
%sql sqlite:///spotify.db

'Connected: @spotify.db'

In [24]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///spotify.db
Done.


type,name,tbl_name,rootpage,sql
table,df_song,df_song,2,"CREATE TABLE ""df_song"" (""track_id"" TEXT, ""track_name"" TEXT, ""track_artist"" TEXT, ""track_popularity"" INTEGER, ""danceability"" REAL, ""energy"" REAL, ""key"" INTEGER, ""loudness"" REAL, ""mode"" INTEGER, ""speechiness"" REAL, ""acousticness"" REAL, ""instrumentalness"" REAL, ""liveness"" REAL, ""valence"" REAL, ""tempo"" REAL, ""duration_ms"" INTEGER)"
table,df_song_album,df_song_album,1041,"CREATE TABLE ""df_song_album"" (""track_id"" TEXT, ""track_album_id"" TEXT)"
table,df_song_playlist,df_song_playlist,1408,"CREATE TABLE ""df_song_playlist"" (""track_id"" TEXT, ""playlist_id"" TEXT)"
table,df_playlist,df_playlist,1826,"CREATE TABLE ""df_playlist"" (""playlist_id"" TEXT, ""playlist_name"" TEXT, ""playlist_subgenre"" TEXT)"
table,df_playlist_genre,df_playlist_genre,1835,"CREATE TABLE ""df_playlist_genre"" (""playlist_subgenre"" TEXT, ""playlist_genre"" TEXT)"
table,df_album,df_album,1836,"CREATE TABLE ""df_album"" (""track_album_id"" TEXT, ""track_album_name"" TEXT, ""track_album_release_date"" TEXT)"


In [25]:
%%sql
SELECT *
FROM df_song
LIMIT 5

 * sqlite:///spotify.db
Done.


track_id,track_name,track_artist,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxury Remix,Ed Sheeran,66,0.748,0.9159999999999999,6,-2.634,1,0.0583,0.102,0.0,0.0653,0.518,122.036,194754
0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,0.726,0.815,11,-4.968999999999999,1,0.0373,0.0724,0.00421,0.35700000000000004,0.693,99.97200000000001,162600
1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,0.675,0.9309999999999999,1,-3.432,0,0.0742,0.0794,2.33e-05,0.11,0.613,124.008,176616
75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,0.718,0.93,7,-3.778,1,0.102,0.0287,9.43e-06,0.204,0.27699999999999997,121.956,169093
1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,0.65,0.833,1,-4.672,1,0.0359,0.0803,0.0,0.0833,0.725,123.976,189052


#### Use an SQL query to find the names of all playlists that contain instrumentals.

From the codebook, `instrumentalness` represents whether a track contains no vocals. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. 

In [26]:
%%sql 
SELECT COUNT(*) AS num_instrumentals, playlist_name
FROM df_song
INNER JOIN df_song_playlist
ON df_song_playlist.track_id = df_song.track_id
INNER JOIN df_playlist
ON df_playlist.playlist_id = df_song_playlist.playlist_id
WHERE instrumentalness > 0.5 
GROUP BY playlist_name
ORDER BY num_instrumentals DESC

 * sqlite:///spotify.db
Done.


num_instrumentals,playlist_name
92,Fitness Workout Electro | House | Dance | Progressive House
91,Lush Lofi
88,Underground Party | Hypnotic | Minimal | Acid | Big Room | Tech | Liquid
81,Jazz Vibes
81,House Electro 2019
78,Lo-Fi Beats
75,Lofi Hip-Hop
60,Selected House
58,💊ELECTRO-HOUSE-TECH💊
54,Sunny Beats


In [27]:
%%sql 
SELECT COUNT(*)
FROM
(SELECT COUNT(*) AS num_instrumentals, playlist_name
FROM df_song
INNER JOIN df_song_playlist
ON df_song_playlist.track_id = df_song.track_id
INNER JOIN df_playlist
ON df_playlist.playlist_id = df_song_playlist.playlist_id
WHERE instrumentalness > 0.5 
GROUP BY playlist_name
HAVING num_instrumentals > 0
ORDER BY num_instrumentals DESC)

 * sqlite:///spotify.db
Done.


COUNT(*)
256


There are 256 playlists that had at least one song that contains instrumentals.

##### check with the original dataframe

In [28]:
np.sum(df[df.playlist_name == 'Lush Lofi'].instrumentalness > 0.5 )

91

This is the same as the result in SQL query. 